<a href="https://colab.research.google.com/github/bjih1999/cifar_img_classify_project/blob/master/cifar_tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from matplotlib import pyplot as plt
 
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
from keras_preprocessing import image
 
cifar10 = datasets.cifar10
# 데이터셋 다운로드 및 준비
(train_images, train_labels), (test_images, test_labels) = cifar10.load_data()
train_images = train_images.reshape((50000, 32, 32, 3))
test_images = test_images.reshape((10000, 32, 32, 3))
 
# 픽셀 값을 0~1 사이로 정규화합니다.
train_images, test_images = train_images / 255.0, test_images / 255.0
 
# 합성곱 층 만들기
# 아래 6줄의 코드에서 Conv2D와 MaxPooling2D 층을 쌓는 일반적인 패턴으로 합성곱 층을 정의합니다.
# CNN은 배치(batch) 크기를 제외하고 (이미지 높이, 이미지 너비, 컬러 채널) 크기의 텐서(tensor)를 입력으로 받습니다.
# MNIST 데이터는 (흑백 이미지이기 때문에) 컬러 채널(channel)이 하나지만 컬러 이미지는 (R,G,B) 세 개의 채널을 가집니다
# 이 예에서는 MNIST 이미지 포맷인 (28, 28, 1) 크기의 입력을 처리하는 CNN을 정의하겠습니다. 이 값을 첫 번째 층의 input_shape 매개변수로 전달합니다.
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)))
# layers.Conv2D(필터, 커널, 활성화함수, 입력데이터의 형태), https://codetorial.net/tensorflow/convolutional_neural_network.html
model.add(layers.MaxPooling2D((2, 2)))
# layers.MaxPooling2D(풀링 필터의 크기)
# Pooling == 합성곱에 의해 얻어진 Feature Map으로부터 값을 샘플링해서 정보를 압축하는 과정
# MaxPooling == 특정 영역에서 가장 큰 값을 샘플링하는 풀링 방식, 예제에서는 2*2로 설정
# strides == 풀링 필터를 이동시키는 각격, 지정해주지 않으면 풀링 필터의 크기와 동일하게 진행, 겹치지 않게 풀링이 진행된다.
 
# model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(layers.Conv2D(32, (3, 3), activation='relu', padding='same'))
model.add(layers.Conv2D(16, (3, 3), activation='relu', padding='same'))
# model.add(layers.Conv2D(8, (3, 3), activation='relu', padding='same'))
 
model.add(tf.keras.layers.GlobalAveragePooling2D())
 
# Dense 층 추가하기
# 모델을 완성하려면 마지막 합성곱 층의 출력 텐서(크기 (4, 4, 64))를 하나 이상의 Dense 층에 주입하여 분류를 수행합니다.
# Dense 층은 벡터(1D)를 입력으로 받는데 현재 출력은 3D 텐서입니다.
# 먼저 3D 출력을 1D로 펼치겠습니다.
model.add(layers.Flatten())
# 그다음 하나 이상의 Dense 층을 그 위에 추가하겠습니다.
model.add(layers.Dense(64, activation='relu'))
# MNIST 데이터는 10개의 클래스가 있으므로 마지막에 Dense 층에 10개의 출력과 소프트맥스 활성화 함수를 사용합니다.
model.add(layers.Dense(10, activation='softmax'))
'''
ayer_names = [layer.name for layer in model.layers]
layer_outputs = [layer.output for layer in model.layers]
 
feature_map_model = tf.keras.models.Model(inputs=model.inputs, outputs=layer_outputs)
image_path= r"cat.jpg"
img = image.load_img(image_path, target_size=(150, 150))
input = image.img_to_array(img)
input = input.reshape((1,) + input.shape)
input = x.reshape((1,) + x.shape)
input /= 255.0
feature_maps = feature_map_model.predict(input)
for layer_name, feature_map in zip(layer_names, feature_maps):
    print(f"The shape of the {layer_name} is =======>> {feature_map.shape}")
'''
# 모델 컴파일 및 훈련
# 옵티마이저, 손실함수, 지표
 
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
 
# 학습시킬 문제와 정답, 훈련 횟수
history = model.fit(train_images, train_labels, epochs=50, validation_split=0.25)
callbacks=[tf.callbacks.EarlyStopping(patience-3, monotpr='val_loss')]
# 모델 평가
# 풀게 할 문제와 정답
test_loss, test_acc = model.evaluate(test_images,  test_labels, verbose=2)
print(test_loss), print(test_acc)
 
# model.summary() 모델 구조 출력
 
 
# cam_model = tf.keras.Model(model.input, outputs=(model.layers[-3].output, model.layers[-1].output))
# gap_weights = model.layers[-1].get_weights()[0]

170500096/170498071 [==============================] - 2s 0us/step
Epoch 1/50
1172/1172 [==============================] - 108s 92ms/step - loss: 1.8483 - accuracy: 0.2837 - val_loss: 1.7548 - val_accuracy: 0.3410
Epoch 2/50
1172/1172 [==============================] - 109s 93ms/step - loss: 1.5643 - accuracy: 0.4028 - val_loss: 1.4769 - val_accuracy: 0.4491
Epoch 3/50
1172/1172 [==============================] - 112s 95ms/step - loss: 1.3973 - accuracy: 0.4795 - val_loss: 1.3255 - val_accuracy: 0.5104
Epoch 4/50
1172/1172 [==============================] - 113s 96ms/step - loss: 1.2877 - accuracy: 0.5282 - val_loss: 1.2460 - val_accuracy: 0.5422
Epoch 5/50
1172/1172 [==============================] - 111s 95ms/step - loss: 1.2133 - accuracy: 0.5544 - val_loss: 1.1681 - val_accuracy: 0.5774
Epoch 6/50
1172/1172 [==============================] - ETA: 0s - loss: 1.1468 - accuracy: 0.5805